In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from matplotlib.ticker import MultipleLocator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Dataset1/TAMIL_NADU_CHENNAI_ENNORE_PORT_13.2567_80.3467_10.681_2020-01-01_2023-12-31_AWS.csv')

In [ ]:
df['WIND_SPEED_10m(kt)'] = df['WIND_SPEED_10m(kt)'] * 0.514444
df = df.rename(columns={'WIND_SPEED_10m(kt)': 'WIND_SPEED_10m(m/s)'})
df['datetime'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME(UTC)'])
df

,DATE,TIME(UTC),RAIN_HOURLY(mm),RAIN_DAILY(mm),TEMP(C),TEMP_MIN(C),TEMP_MAX(C),TEMP_MIN_MAX_DAY(C),DEW_POINT(C),RH(%),RH_MIN_MAX_DAY(%),WIND_DIR_10m(deg),WIND_SPEED_10m(m/s),WIND_MAX/GUST_10m(kt),SLP(hPa),MSLP(hPa),SUNSHINE(HH:MM),datetime
0,1/1/2020,0:00:00,1.0,1.0,25.5,25.5,26.4,NaN,25.5,100.0,NaN,57.0,3.086664,NaN,1011.6,1012.8,0.0,2020-01-01 00:00:00
1,1/1/2020,0:20:00,0.0,1.0,25.6,25.5,25.6,NaN,25.6,100.0,NaN,49.0,2.057776,NaN,1011.8,1013.0,0.0,2020-01-01 00:20:00
2,1/1/2020,0:40:00,0.0,1.0,25.8,25.5,25.8,NaN,25.8,100.0,NaN,41.0,2.057776,NaN,1012.0,1013.2,0.0,2020-01-01 00:40:00
3,1/1/2020,1:00:00,0.0,1.0,25.9,25.5,25.9,NaN,25.9,100.0,NaN,54.0,2.572220,NaN,1012.3,1013.5,0.0,2020-01-01 01:00:00
4,1/1/2020,1:20:00,0.0,1.0,26.0,25.9,26.0,NaN,26.0,100.0,NaN,41.0,3.086664,NaN,1012.6,1013.8,0.0,2020-01-01 01:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106158,12/31/2023,22:45:00,NaN,0.0,22.0,21.6,26.7,NaN,19.5,86.0,NaN,274.0,1.028888,4.0,1013.3,1014.5,NaN,2023-12-31 22:45:00
106159,12/31/2023,23:00:00,NaN,0.0,21.9,21.6,26.7,NaN,20.0,89.0,NaN,268.0,1.028888,4.0,1013.3,1014.5,NaN,2023-12-31 23:00:00
106160,12/31/2023,23:15:00,NaN,0.0,22.1,21.6,26.7,NaN,20.0,88.0,NaN,277.0,1.543332,6.0,1013.4,1014.6,NaN,2023-12-31 23:15:00
106161,12/31/2023,23:30:00,NaN,0.0,22.3,21.6,26.7,NaN,19.8,86.0,NaN,283.0,2.057776,6.0,1013.6,1014.8,NaN,2023-12-31 23:30:00


In [ ]:
os.makedirs("Avg_Speed_Ennore", exist_ok=True)

In [ ]:
from datetime import datetime  # Import
start_date = datetime(2020,1,1)
end_date = datetime(2023, 12, 31)

current_date = start_date
while current_date <= end_date:
    # Filter data for the current month
    month_data = df[(df['datetime'].dt.year == current_date.year) & (df['datetime'].dt.month == current_date.month)]

    # Group by time and calculate the average wind speed
    month_data_avg = month_data.groupby(month_data['datetime'].dt.time)['WIND_SPEED_10m(m/s)'].mean().reset_index()

    # Save the average wind speed data to a CSV file
    month_name = current_date.strftime("%Y_%B")
    file_name = os.path.join("Avg_Speed_Ennore", f"{month_name}_wind_speed.csv")
    month_data_avg.to_csv(file_name, index=False)

    # Move to the next month
    current_date = current_date.replace(day=1) + pd.DateOffset(months=1)

In [ ]:
def process_csv_files(folder_path):

    # Create a new folder for the FFT plots
    output_folder = os.path.join(folder_path, 'fft_plots')
    os.makedirs(output_folder, exist_ok=True)

    # List all csv files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)

        # Read the CSV file
        data = pd.read_csv(file_path)

        # Assuming the column for wind speed is named 'WindSpeed'
        wind_speed = data['WIND_SPEED_10m(m/s)'].values

        # Number of samples
        N = len(wind_speed)

        # Sampling interval (15 minutes)
        T = 15 * 60  # in seconds

        # Perform FFT
        yf = fft(wind_speed)
        xf = fftfreq(N, T)[:N//2]

        # Calculate amplitude
        amplitude = 2.0 / N * np.abs(yf[:N//2])

        # Define the frequency range to be extracted (15 min to 780 min)
        min_period = 15 * 60  # 15 minutes in seconds
        max_period = 13 * 60 * 60  # 13 hours in seconds

        min_freq = 1 / max_period  # corresponding frequency for max period
        max_freq = 1 / min_period  # corresponding frequency for min period

        # Filter frequencies within the desired range
        mask = (xf >= min_freq) & (xf <= max_freq)
        xf_filtered = xf[mask]
        amplitude_filtered = amplitude[mask]

        # Plot frequency vs amplitude
        plt.figure(figsize=(10, 6))
        plt.plot(xf_filtered, amplitude_filtered)
        plt.title('Frequency vs Amplitude')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')

        # Add more grid lines
        plt.grid(which='both', axis='both', linestyle='--', linewidth=0.5)

        # Customize major and minor ticks
        major_locator = MultipleLocator(0.0001)
        minor_locator = MultipleLocator(0.000025)

        ax = plt.gca()
        ax.xaxis.set_major_locator(major_locator)
        ax.xaxis.set_minor_locator(minor_locator)
        ax.minorticks_on()

        # Save the plot
        plot_file_path = os.path.join(output_folder, csv_file.replace('.csv', '.png'))
        plt.savefig(plot_file_path)
        plt.close()


In [ ]:
# Example usage
folder_path = r"/content/Avg_Speed_Goa"
process_csv_files(folder_path)

In [ ]:
def process_combined_csv_files(folder_path, selected_months):
    # Create a new folder for the combined FFT plots
    output_folder = os.path.join(folder_path, 'fft_plots_combined')
    os.makedirs(output_folder, exist_ok=True)

    # List all csv files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Select the CSV files for the months you want to combine
    selected_csv_files = [f for f in csv_files if any(month in f for month in selected_months)]

    # Extract wind speed data for the selected months
    wind_speed_data_list = []
    for csv_file in selected_csv_files:
        file_path = os.path.join(folder_path, csv_file)
        data = pd.read_csv(file_path)
        wind_speed_data_list.append(data['WIND_SPEED_10m(m/s)'].values)

    # Concatenate all wind speed data
    combined_wind_speed_data = np.concatenate(wind_speed_data_list)

    # Number of samples
    N = len(combined_wind_speed_data)

    # Sampling interval (15 minutes)
    T = 15 * 60  # in seconds

    # Perform FFT
    yf = fft(combined_wind_speed_data)
    xf = fftfreq(N, T)[:N//2]

    # Calculate amplitude
    amplitude = 2.0 / N * np.abs(yf[:N//2])

    # Define the frequency range to be extracted (15 min to 780 min)
    min_period = 15 * 60  # 15 minutes in seconds
    max_period = 13 * 60 * 60  # 13 hours in seconds

    min_freq = 1 / max_period  # corresponding frequency for max period
    max_freq = 1 / min_period  # corresponding frequency for min period

    # Filter frequencies within the desired range
    mask = (xf >= min_freq) & (xf <= max_freq)
    xf_filtered = xf[mask]
    amplitude_filtered = amplitude[mask]

    # Plot frequency vs amplitude
    plt.figure(figsize=(10, 6))
    plt.plot(xf_filtered, amplitude_filtered)
    plt.title('Combined FFT: Frequency vs Amplitude')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')

    # Add more grid lines
    plt.grid(which='both', axis='both', linestyle='--', linewidth=0.5)

    # Customize major and minor ticks
    major_locator = MultipleLocator(0.0001)
    minor_locator = MultipleLocator(0.000025)

    ax = plt.gca()
    ax.xaxis.set_major_locator(major_locator)
    ax.xaxis.set_minor_locator(minor_locator)
    ax.minorticks_on()

    # Save the plot in the 'fft_plots_combined' folder
    plot_file_path = os.path.join(output_folder, 'Combined_FFT.png')
    plt.savefig(plot_file_path)
    plt.close()

# Example usage
folder_path = r"/content/Avg_Speed_Ennore"
selected_months = ['2020_September', '2020_October', '2020_November']  # Add or change the months as needed
process_combined_csv_files(folder_path, selected_months)

In [ ]:
def process_combined_csv_files(folder_path, season_months, season_name, year):
    # Create a new folder for the combined FFT plots for each year
    output_folder = os.path.join(folder_path, f'fft_plots_{season_name}_{year}')
    os.makedirs(output_folder, exist_ok=True)

    # List all csv files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Select the CSV files for the months you want to combine for the specified year
    selected_csv_files = [f for f in csv_files if any(month in f for month in season_months) and str(year) in f]

    # Extract wind speed data for the selected months
    wind_speed_data_list = []
    for csv_file in selected_csv_files:
        file_path = os.path.join(folder_path, csv_file)
        data = pd.read_csv(file_path)
        wind_speed_data_list.append(data['WIND_SPEED_10m(m/s)'].values)

    if not wind_speed_data_list:
        print(f"No data available for {season_name} {year}")
        return

    # Concatenate all wind speed data
    combined_wind_speed_data = np.concatenate(wind_speed_data_list)

    # Number of samples
    N = len(combined_wind_speed_data)

    # Sampling interval (15 minutes)
    T = 15 * 60  # in seconds

    # Perform FFT
    yf = fft(combined_wind_speed_data)
    xf = fftfreq(N, T)[:N//2]

    # Calculate amplitude
    amplitude = 2.0 / N * np.abs(yf[:N//2])

    # Define the frequency range to be extracted (15 min to 780 min)
    min_period = 15 * 60  # 15 minutes in seconds
    max_period = 13 * 60 * 60  # 13 hours in seconds

    min_freq = 1 / max_period  # corresponding frequency for max period
    max_freq = 1 / min_period  # corresponding frequency for min period

    # Filter frequencies within the desired range
    mask = (xf >= min_freq) & (xf <= max_freq)
    xf_filtered = xf[mask]
    amplitude_filtered = amplitude[mask]

    # Plot frequency vs amplitude
    plt.figure(figsize=(10, 6))
    plt.plot(xf_filtered, amplitude_filtered)
    plt.title(f'Combined FFT: Frequency vs Amplitude ({season_name} {year})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')

    # Add more grid lines
    plt.grid(which='both', axis='both', linestyle='--', linewidth=0.5)

    # Customize major and minor ticks
    major_locator = MultipleLocator(0.0001)
    minor_locator = MultipleLocator(0.000025)

    ax = plt.gca()
    ax.xaxis.set_major_locator(major_locator)
    ax.xaxis.set_minor_locator(minor_locator)
    ax.minorticks_on()

    # Save the plot in the corresponding season folder
    plot_file_path = os.path.join(output_folder, f'Combined_FFT_{season_name}_{year}.png')
    plt.savefig(plot_file_path)
    plt.close()

# Example usage
folder_path = r"/content/Avg_Speed_Ennore"

# Define the seasons and their corresponding months
seasons = {
    'Winter': ['_January', '_February'],
    'Pre-monsoon': ['_March', '_April', '_May'],
    'Monsoon': ['_June', '_July', '_August', '_September'],
    'Post-monsoon': ['_October', '_November', '_December']
}

# List all available years in the folder
years = sorted({int(f.split('_')[0]) for f in os.listdir(folder_path) if f.endswith('.csv')})

# Process each season for each year
for year in years:
    for season_name, season_months in seasons.items():
        process_combined_csv_files(folder_path, season_months, season_name, year)


ValueError: invalid number of data points (0) specified

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from matplotlib.ticker import MultipleLocator

def process_combined_csv_files(folder_path, season_months, season_name, year, output_folder):
    # List all csv files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Select the CSV files for the months you want to combine for the specified year
    selected_csv_files = [f for f in csv_files if any(month in f for month in season_months) and str(year) in f]

    # Extract wind speed data for the selected months
    wind_speed_data_list = []
    for csv_file in selected_csv_files:
        file_path = os.path.join(folder_path, csv_file)
        data = pd.read_csv(file_path)
        wind_speed_data_list.append(data['WIND_SPEED_10m(m/s)'].values)

    if not wind_speed_data_list:
        print(f"No data available for {season_name} {year}")
        return

    # Concatenate all wind speed data
    combined_wind_speed_data = np.concatenate(wind_speed_data_list)

    # Number of samples
    N = len(combined_wind_speed_data)

    # Sampling interval (15 minutes)
    T = 15 * 60  # in seconds

    # Perform FFT
    yf = fft(combined_wind_speed_data)
    xf = fftfreq(N, T)[:N//2]

    # Calculate amplitude
    amplitude = 2.0 / N * np.abs(yf[:N//2])

    # Define the frequency range to be extracted (15 min to 780 min)
    min_period = 15 * 60  # 15 minutes in seconds
    max_period = 13 * 60 * 60  # 13 hours in seconds

    min_freq = 1 / max_period  # corresponding frequency for max period
    max_freq = 1 / min_period  # corresponding frequency for min period

    # Filter frequencies within the desired range
    mask = (xf >= min_freq) & (xf <= max_freq)
    xf_filtered = xf[mask]
    amplitude_filtered = amplitude[mask]

    # Plot frequency vs amplitude
    plt.figure(figsize=(10, 6))
    plt.plot(xf_filtered, amplitude_filtered)
    plt.title(f'Combined FFT: Frequency vs Amplitude ({season_name} {year})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')

    # Add more grid lines
    plt.grid(which='both', axis='both', linestyle='--', linewidth=0.5)

    # Customize major and minor ticks
    major_locator = MultipleLocator(0.0001)
    minor_locator = MultipleLocator(0.000025)

    ax = plt.gca()
    ax.xaxis.set_major_locator(major_locator)
    ax.xaxis.set_minor_locator(minor_locator)
    ax.minorticks_on()

    # Save the plot in the single output folder
    plot_file_path = os.path.join(output_folder, f'Combined_FFT_{season_name}_{year}.png')
    plt.savefig(plot_file_path)
    plt.close()

# Example usage
folder_path = r"/content/Avg_Speed_Ennore"

# Define the seasons and their corresponding months
seasons = {
    'Winter': ['_January', '_February'],
    'Pre-monsoon': ['_March', '_April', '_May'],
    'Monsoon': ['_June', '_July', '_August', '_September'],
    'Post-monsoon': ['_October', '_November', '_December']
}

# Create a single folder for all FFT plots
output_folder_drive = '/content/drive/MyDrive/Percentile'
os.makedirs(output_folder_drive, exist_ok=True)

# List all available years in the folder
years = sorted({int(f.split('_')[0]) for f in os.listdir(folder_path) if f.endswith('.csv')})

# Process each season for each year
for year in years:
    for season_name, season_months in seasons.items():
        process_combined_csv_files(folder_path, season_months, season_name, year, output_folder_drive)